# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part V: Evaluating the Model on the Test Set

In this final part of the tutorial, we will reload the model developed in Part IV and test it on the test `CandidateSet`.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass, and load the test set:

In [ ]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

## Automatically Creating Features
We also generate features for the test set; note that we specify `create_new_keyset=False` to use the features from the training set.  For further details see _Intro Tutorial 4_.

In [ ]:
from snorkel.annotations import FeatureAnnotator

featurizer = FeatureAnnotator()
%time featurizer.apply(split=2, replace_key_set=False)

If we've already computed the features, we can just use the below step

In [ ]:
from snorkel.annotations import load_feature_matrix
F_test = load_feature_matrix(session, split=2)
F_test

## Reloading the Discriminative Model

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
%time disc_model.load(session)

## Evaluating on the Test `CandidateSet`

First, we load the test set labels and gold candidates we made in Part III.

In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

In [ ]:
tp, fp, tn, fn = disc_model.score(session, F_test, L_gold_test)

## Viewing Examples
We can also view the results on the test `CandidateSet`,  You can view the true positives, false positives, true negatives, and false negatives using the `Viewer`.

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(tp, session, annotator_name="Tutorial Part V User")
else:
    sv = None

In [ ]:
sv

You've completed the introduction to Snorkel!